<a href="https://colab.research.google.com/github/anuj0456/courses/blob/master/competitions/he_holiday_season_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # !pip install tf-nightly
# !pip uninstall tensorflow  
# !pip install tensorflow

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import os
import csv
import sys
import cv2
import time
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn import preprocessing
from keras.models import Sequential
from tensorflow.keras import layers
from keras.optimizers import Adam, SGD
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import Xception, InceptionResNetV2, EfficientNetB4, ResNet50

In [4]:
# !unzip test.zip
# !unzip train.zip

In [5]:
IMG_SIZE = 256
batch_size = 32
epoch = 5
splits = 5
TRAIN_PATH = '/content/train/'
TEST_PATH = '/content/test/'
csv = '/content/train.csv'

In [6]:
df = pd.read_csv(csv)

In [7]:
kfold = KFold(splits, True)
# enumerate splits

In [8]:
train_img = []
x = 0
for img_name in tqdm(df['Image'].values):
    img = cv2.imread(TRAIN_PATH + img_name)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    train_img.append(img)
    
x_train = np.array(train_img, np.float32) / 255
print(x_train.shape)

100%|██████████| 6469/6469 [00:02<00:00, 2175.81it/s]


(6469, 256, 256, 3)


In [9]:
label_list = df['Class'].tolist()
label_numeric = {k: v for v, k in enumerate(set(label_list))}
y_train = [label_numeric[k] for k in label_list]
y_train = np.array(y_train)

In [10]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
print(y_train.shape)

(6469, 6)


In [11]:
train_gen = ImageDataGenerator(rotation_range=30, 
                                width_shift_range=0.1, 
                                height_shift_range=0.1,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                )
train_gen.fit(x_train)

In [12]:
rms = RMSprop(lr = 0.001, momentum=0.9, clipnorm = 1., clipvalue = 1.)
adam = Adam(lr = 1e-4)
sgd = SGD(lr = 1e-4, momentum=0.9)

In [13]:
def get_model1():
    core = EfficientNetB4(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [14]:
model1 = get_model1()
model1.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

71688192/71686520 [==============================] - 1s 0us/step


In [15]:
model_save1 = ModelCheckpoint("/content/model_1", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop1 = EarlyStopping(monitor = 'loss', min_delta = 0.01, 
                           patience = 2, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr1 = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, 
                              patience = 2, min_delta = 0.01, 
                              mode = 'min', verbose = 1)

In [16]:
# for train, test in kfold.split(x_train):
history1 = model1.fit(train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch,callbacks=[model_save1])

Epoch 1/5
203/203 [==============================] - 150s 631ms/step - loss: 1.4022 - accuracy: 0.5380

Epoch 00001: loss improved from inf to 1.07544, saving model to /content/model_1
Epoch 2/5
203/203 [==============================] - 128s 628ms/step - loss: 0.6752 - accuracy: 0.7516

Epoch 00002: loss improved from 1.07544 to 0.63396, saving model to /content/model_1
Epoch 3/5
203/203 [==============================] - 128s 630ms/step - loss: 0.4924 - accuracy: 0.8252

Epoch 00003: loss improved from 0.63396 to 0.48630, saving model to /content/model_1
Epoch 4/5
203/203 [==============================] - 129s 631ms/step - loss: 0.4072 - accuracy: 0.8538

Epoch 00004: loss improved from 0.48630 to 0.39540, saving model to /content/model_1
Epoch 5/5
203/203 [==============================] - 129s 631ms/step - loss: 0.3381 - accuracy: 0.8777

Epoch 00005: loss improved from 0.39540 to 0.33690, saving model to /content/model_1


In [17]:
def get_model2():
    core = InceptionResNetV2(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [18]:
model2 = get_model2()
model2.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

219062272/219055592 [==============================] - 5s 0us/step


In [19]:
model_save2 = ModelCheckpoint("/content/model_2", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop2 = EarlyStopping(monitor = 'loss', min_delta = 0.01, 
                           patience = 2, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr2 = ReduceLROnPlateau(monitor = 'loss', factor = 0.1, 
                              patience = 2, min_delta = 0.01, 
                              mode = 'min', verbose = 1)

In [ ]:
history2 = model2.fit(train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save2])

Epoch 1/5
203/203 [==============================] - 109s 439ms/step - loss: 1.6346 - accuracy: 0.4593

Epoch 00001: loss improved from inf to 1.26764, saving model to /content/model_2
Epoch 2/5
197/203 [============================>.] - ETA: 2s - loss: 0.8360 - accuracy: 0.7092

In [ ]:
def get_model3():
    core = Xception(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [ ]:
model3 = get_model3()
model3.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
model_save3 = ModelCheckpoint("/content/model_3", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop3 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr3 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.001, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [ ]:
history3 = model3.fit( train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save3])

In [ ]:
def get_model4():
    core = ResNet50(include_top=False, input_tensor=None,weights='imagenet',input_shape=(IMG_SIZE, IMG_SIZE,3))
    model = Sequential()
    model.add(core)
    model.add(Flatten())
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(6, activation='softmax'))
    
    return model

In [ ]:
model4 = get_model4()
model4.compile(optimizer= adam, metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
model_save4 = ModelCheckpoint("/content/model_4", 
                             save_best_only = True, 
                             save_weights_only = True,
                             monitor = 'loss', 
                             mode = 'min', verbose = 1)
early_stop4 = EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                           patience = 5, mode = 'min', verbose = 1,
                           restore_best_weights = True)
reduce_lr4 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.001, 
                              patience = 2, min_delta = 0.001, 
                              mode = 'min', verbose = 1)

In [ ]:
history4 = model4.fit( train_gen.flow(x_train, y_train, batch_size=batch_size), epochs=epoch, callbacks=[model_save4])

In [ ]:
try:
  os.remove('/content/submission.csv')
except Exception as e:
  print("No file exists")
with open('/content/submission.csv', 'w', newline='') as file:
      writer = csv.writer(file, delimiter = ',')
      writer.writerow(["Image", "Class"])

In [ ]:
members = [model1, model2, model3, model4]

In [ ]:
def ensemble_predictions(members, testX):
	# make predictions
	yhats = [model.predict(testX) for model in members]
	yhats = np.array(yhats)
	summed = np.sum(yhats, axis=0)
	return summed

In [ ]:
test_images = os.listdir(TEST_PATH)
test_img = []
for img_name in tqdm(test_images):
    img = cv2.imread(TEST_PATH + img_name)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    test_img.append(img)

In [ ]:
x_test = np.array(test_img, np.float32) / 255

In [ ]:
predictions = ensemble_predictions(members,x_test)

In [ ]:
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in label_numeric.items()}
pred_labels = [rev_y[k] for k in predictions]

In [ ]:
sub = pd.DataFrame({'Image': test_images, 'Class': pred_labels})
sub.to_csv('/content/submission.csv', index = False)

In [ ]:
from google.colab import files
files.download('/content/submission.csv')

In [ ]:
import gc 
gc.collect()